In [1]:
import pandas as pd
import numpy as np
import re
import math
import tabula
import camelot
import pdfminer
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
try:
    from cStringIO import StringIO
except ImportError:
    from io import StringIO 
import matplotlib.pyplot as plt 

In [14]:
def abc(a):
    if type(a) ==str:
        if len(a.split(' '))==2:
            z=a.split(' ')[1]
        else:
            z=a.split(' ')[0]
    else :
        z=a
    return z

def isnan(value):
    try:
        return math.isnan(float(value))
    except:
        return False

def backfill(df, narr):
    print("backfill")
    df = df[pd.notnull(df[narr])]
    df.iloc[:,-1]=df.iloc[:,-1].fillna(df.iloc[:,-2])
    
    try:
        bal=[c for c in df.columns if "BALANCE" in str(c).upper()][0]
        df[bal]=df[bal].apply(lambda x: abc(x))
    except:
        print("\nBalance Column Missing")
        
    df["flag"]=df.iloc[:,-1].shift(1)
    df=df[~df.index.isin(df[df.iloc[:,-1]==df.iloc[:,-2]].index)]

    df=df.T.drop_duplicates().T
    df[bal]=df[bal].apply(lambda x: str(x).replace(",","").replace("(Cr)","").replace("(Dr)","")).astype(float)
    df["flag"]=df.iloc[:,0].astype(str)+df[bal].astype(str)
    df["flag2"]=np.arange(len(df))

    df.loc[df[["flag"]].duplicated(keep=False), 'flag'] = df["flag"]+df["flag2"].astype(str)
    df['flag']=df['flag'].apply(lambda row : np.nan if 'nannan' in row else row).fillna(method='bfill')

    df[narr]=df.groupby('flag')[narr].transform(lambda x:''.join(x))
    df=df.drop_duplicates(['flag'],keep='last').iloc[0:,:-2].reset_index(drop=True)
    df.to_csv("parsed_completed.csv",index=0)
    print("parsed_completed")

def proceed(df):
    try:
        idx=[ c for c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('date').any(), axis=1) ==True].index ][0]
        df.columns=df.iloc[idx] ; df=df.iloc[idx+1:,:] ; df.reset_index(drop=True,inplace=True)
        print("1st attempt")
    except:
        try:
            idx=[ c for c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('particular').any(), axis=1) ==True].index ][0]
            df.columns=df.iloc[idx] ; df=df.iloc[idx+1:,:] ; df.reset_index(drop=True,inplace=True)           
        except:
            print("\nAxis-Column headers missing"); pass

    try:
        df = df.drop(["Init."], axis=1)
    except:
        try:
            df = df.drop(["Branch Name"], axis=1)
        except:pass

    df=df[~df.index.isin(df[df.apply(lambda row: row.astype(str).str.lower().str.contains('opening balance|transaction total|closing balance|particulars|transaction').any(), axis=1) ==True].index)]
    #df.drop(df.nunique(dropna=False)[(df.nunique(dropna=False) == 1)].index, axis=1,inplace=True)

    try:
        narr=[n for n in df.columns if "PARTICULARS" in str(n).upper()][0]
    except:
        try:
            narr=[n for n in df.columns if "NARRATION" in str(n).upper()][0]
        except:
            try:
                narr=[n for n in df.columns if "DESCRIPTION" in str(n).upper()][0] 
            except:
                pass

    list1 = ['Tran Date','Chq No','Particulars','Debit','Credit','Balance']
    list2 = ['Tran Date','Value Date','Transaction Particulars','Chq No','Amount(INR)','DR/CR','Balance(INR)']
    list3 = ['Tran Date','Value Date','Transaction Particulars','Chq No.','Amount','DR|CR','Balance']

    if list(df.columns.values) == list1 :
        print("list1")
        backfill(df,narr)
        
    elif list(df.columns.values) == list2 :
        print("list2")
        backfill(df,narr)

    elif list(df.columns.values) == list3:
        print("list3")   
        
        try:
            bal=[c for c in df.columns if "BALANCE" in str(c).upper()][0]
            df[bal]=df[bal].apply(lambda x: abc(x))
        except:
            print("\nBalance Column Missing")

        try:    
            df.replace(r'^\s*$', np.nan, regex=True, inplace = True)
            #type(df["Transaction Particulars"][15])
        except:pass

        try:
            df.dropna(axis=0, how='all',inplace=True)
        except:pass

        try:
            df.reset_index(drop=True,inplace=True)
        except:pass

        try:
            for j,i in enumerate(df["Transaction Particulars"]):
                if isnan(i) == True:
                    df[narr][j] = df[narr][j-1]+df[narr][j+1]
                else:
                    pass
        except:pass

        try:
            df.dropna(subset=['Tran Date'], inplace = True)
        except:
            try:
                df.dropna(subset=['Value Date'], inplace = True)
            except:pass

        df.to_csv("parsed_completed.csv",index=0)
        print("parsed_completed")
        
    else:
        print("In Proceed, Another pattern found")

def came(f):
    tables = camelot.read_pdf(f, flavor='stream', pages='1,2', edge_tol=500)
    if len(tables) != 0:
        dt=pd.DataFrame(); tmp=pd.DataFrame()
        first_check = tables[0].df; second_check = tables[1].df;
        # print(first_check.shape[1])
        # print(second_check.shape[1])
        if (first_check.shape[1] > 5):
            try:
                idx=[ c for c in first_check[first_check.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in first_check[first_check.apply(lambda row: row.astype(str).str.lower().str.contains('date').any(), axis=1) ==True].index ][0]
                first_check.columns=first_check.iloc[idx]
            except:
                try:
                    idx=[ c for c in first_check[first_check.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in dt[dt.apply(lambda row: row.astype(str).str.lower().str.contains('particular').any(), axis=1) ==True].index ][0]
                    first_check.columns=first_check.iloc[idx]
                except:
                    print("\nAxis-Column headers missing")
            tables2 = camelot.read_pdf(f, flavor='stream', pages='all', edge_tol=500)
            print("\nlength of first_check:",len(first_check.columns))
            if len(first_check.columns) == 7:
                print("Column length is 7")
                for i in range(len(tables2)):
                    tmp=tables2[i].df
                    if (tmp.shape[1] < 5):
                        print("\nDiiferent structure on page:",i); pass
                    elif (tmp.shape[1] == 7):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        dt = pd.concat([dt,tmp]).reset_index(drop=True)
                    elif (tmp.shape[1] == 6):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        tmp[6] = np.nan; tmp = tmp[[0,6,1,2,3,4,5]]; tmp.columns = range(tmp.shape[1]);
                        dt = pd.concat([dt,tmp]).reset_index(drop=True)
                    else:
                        print("\nAnother Structure on page:",i); pass

            elif len(first_check.columns) == 8:
                print("Column length is 8")
                for i in range(len(tables2)):
                    tmp=tables2[i].df   
                    if (tmp.shape[1] < 5):
                        print("\nDiiferent structure on page:",i); pass
                    elif (tmp.shape[1] == 8):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        dt = pd.concat([dt,tmp]).reset_index(drop=True)
                    elif (tmp.shape[1] == 7):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        tmp[7] = np.nan; tmp = tmp[[0,1,2,7,3,4,5,6]]; tmp.columns = range(tmp.shape[1]);
                        dt = pd.concat([dt,tmp]).reset_index(drop=True)
                    else:
                        print("\nAnother Structure on page:",i); pass
            elif len(first_check.columns) > 8:
                print("Column length greater than 8")
                for i in range(len(tables2)):
                    tmp=tables2[i].df 
                    if (tmp.shape[1] > 8):
                        try:
                            idx=[ c for c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('date').any(), axis=1) ==True].index ][0]
                            tmp.columns=tmp.iloc[idx] ; tmp=tmp.iloc[idx:,:] ; tmp.reset_index(drop=True,inplace=True)
                        except:
                            try:
                                idx=[ c for c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('particular').any(), axis=1) ==True].index ][0]
                                tmp.columns=tmp.iloc[idx] ; tmp=tmp.iloc[idx:,:] ; tmp.reset_index(drop=True,inplace=True)           
                            except:
                                print("\nAxis-Column headers missing"); pass
                        tmp.drop("",axis = 1,inplace = True)
                        tmp.columns = [0,1,2,3,4,5,6,7]
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        # print("\n",tmp)
                        dt=pd.concat([dt,tmp]).reset_index(drop=True)
                    elif (tmp.shape[1] == 8):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        # print("\n",tmp)
                        dt = pd.concat([dt,tmp]).reset_index(drop=True)
                    elif (tmp.shape[1] == 7):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        tmp[7] = np.nan; tmp = tmp[[0,1,2,7,3,4,5,6]]; tmp.columns = range(tmp.shape[1]);
                        # print("\n",tmp)
                        dt = pd.concat([dt,tmp]).reset_index(drop=True)
                    elif (tmp.shape[1] == 6):
                        try:
                            idx=[ c for c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('date').any(), axis=1) ==True].index ][0]
                        except:
                            try:
                                idx=[ c for c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('particular').any(), axis=1) ==True].index ][0]
                            except:
                                print("\nAxis-Column headers missing")
                        tmp.columns=tmp.iloc[idx] ; tmp=tmp.iloc[idx+1:,:] ; tmp.reset_index(drop=True,inplace=True) 
                        tmp=tmp[~tmp.index.isin(tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('page:|account status|total|reason for return|inward clg|opening balance|statement of a/c').any(), axis=1) ==True].index)]
                        if tmp.columns[0] == 'Tran Date\nValue Date\nTransaction Particulars':
                            tmp.columns = ["A","B","C","D","E","F"]
                            tmp.reset_index(drop = True, inplace=True)
                            tmp1 = pd.DataFrame(tmp.A.str.split('\n',2).tolist(),columns=["Z1", "Z2", "Z3"])
                            result = pd.concat([tmp1, tmp], axis=1, sort=False)
                            result.drop('A' ,axis=1, inplace=True)
                            result["Y"] = result["Z1"]
                            for i,j in enumerate(result["Y"]):
                                if re.search(r'(\d+-\d+-\d+)',j):
                                    result.at[i, 'Y'] = ""
                                    pass
                                elif re.search(r'^\s*$',j):
                                    result.at[i, 'Y'] = ""
                                    pass
                                else:
                                    pass
                                    
                            for i,j in enumerate(result["Z1"]):
                                if re.search(r'(\d+-\d+-\d+)',j):
                                    pass
                                elif re.search(r'^\s*$',j):
                                    result.at[i, 'Z1'] = ""
                                    pass
                                else:
                                    result.at[i, 'Z1'] = ""
                                    pass 
                            result.fillna(value=pd.np.nan, inplace=True)
                            result['X'] = result['Z3'].fillna('') + result['Y'].fillna('')
                            result.drop(['Z3','Y'] ,axis=1, inplace=True)
                            result = result[['Z1', 'Z2', 'X', 'B', 'C','D','E','F']]
                            result.columns = [0,1,2,3,4,5,6,7]
                            result=pd.DataFrame(result).replace(r'^\s*$', np.nan, regex=True)
                            # print("\n",result)
                            dt=pd.concat([dt,result]).reset_index(drop=True)
                        else:
                            print("\nIn without box pdf another Structure on page:",i); pass

        elif(second_check.shape[1] > 5):
            try:
                idx=[ c for c in second_check[second_check.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in second_check[second_check.apply(lambda row: row.astype(str).str.lower().str.contains('date').any(), axis=1) ==True].index ][0]
                second_check.columns=second_check.iloc[idx]
            except:
                try:
                    idx=[ c for c in second_check[second_check.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in dt[dt.apply(lambda row: row.astype(str).str.lower().str.contains('particular').any(), axis=1) ==True].index ][0]
                    second_check.columns=second_check.iloc[idx]
                except:
                    print("\nAxis-Column headers missing")
            tables2 = camelot.read_pdf(f, flavor='stream', pages='all', edge_til=500)
            if len(second_check.columns) == 7:
                print("Column length is 7")
                for i in range(len(tables2)):
                    tmp=tables2[i].df
                    if (tmp.shape[1] < 5):
                        print("\nDiiferent structure on page:",i); pass
                    elif (tmp.shape[1] == 7):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        dt = pd.concat([dt,tmp]).reset_index(drop=True)
                    elif (tmp.shape[1] == 6):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        tmp[6] = np.nan; tmp = tmp[[0,6,1,2,3,4,5]]; tmp.columns = range(tmp.shape[1]);
                        dt = pd.concat([dt,tmp]).reset_index(drop=True)
                    else:
                        print("\nAnother Structure on page:",i); pass

            elif len(second_check.columns) == 8:
                print("Column length is 8")
                for i in range(len(tables2)):
                    #print("\n",i,":",tmp.shape[1])
                    tmp=tables2[i].df
                    if (tmp.shape[1] < 5):
                        print("\nDiiferent structure on page:",i); pass
                    elif (tmp.shape[1] == 8):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        dt = pd.concat([dt,tmp]).reset_index(drop=True)
                    elif (tmp.shape[1] == 7):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        tmp[7] = np.nan; tmp = tmp[[0,1,2,7,3,4,5,6]]; tmp.columns = range(tmp.shape[1]);
                        dt = pd.concat([dt,tmp]).reset_index(drop=True)
                    else:
                        print("\nAnother Structure on page:",i); pass

    else:
        print("\nLength of table is 0"); pass   
    return dt

try:
    f = r'C:\Users\MudraCircle\Desktop\bks_raw\Parsing_testing\Axis_new\files\axis07.pdf'
    try:
        df = came(f); df = proceed(df);
        print("Completed.")
    except:
        print("Not parsed")
        pass
except:
    print("\n Can't parse") 

Column length is 7

Diiferent structure on page: 0

Diiferent structure on page: 6

Diiferent structure on page: 7
1st attempt
list1
backfill


c:\users\mudracircle\appdata\local\programs\python\python36\lib\site-packages\pandas\core\indexing.py:576: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value
c:\users\mudracircle\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\mudracircle\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a c

parsed_completed
Completed.
